# Notebook 1 — Pricing Calls & Puts

This notebook demonstrates the core pricing functionality of `optpricer`. We show how to:

1. Define an option contract using `OptionSpec` (an immutable dataclass)
2. Price European options with **Black-Scholes**, **Monte Carlo**, and **Binomial Trees**
3. Compute **Greeks** and recover **implied volatility**
4. Work with immutable specs — clone-and-tweak via `dataclasses.replace`
5. Price under **stochastic volatility** (Heston, SABR), **jump-diffusion** (Merton), and **local volatility** models

In [1]:
import optpricer as op
from dataclasses import replace, asdict

## 1. Setup & Quick Start

`OptionSpec` is a frozen dataclass that bundles all contract and market parameters into a single, hashable object. Because it is immutable, you can safely pass it around without worrying about accidental mutation.

In [2]:
opt = op.OptionSpec(S0=100, K=100, T=1.0, r=0.03, sigma=0.20)
print("asdict:", asdict(opt))

opt2 = replace(opt, K=110)
print("BS call K=110:", op.bs_price(opt2))

asdict: {'S0': 100, 'K': 100, 'T': 1.0, 'r': 0.03, 'sigma': 0.2, 'q': 0.0}
BS call K=110: 5.2933980580449


We define an ATM option ($S_0 = K = 100$) with 1-year maturity, 3% risk-free rate, and 20% volatility. The `replace()` helper from `dataclasses` lets us create a variant with a different strike without mutating the original.

In [3]:
# Monte Carlo should be very close to BS (within ~0.5% with enough paths)
mc = op.euro_price_mc(opt2, op.CALL, n_paths=100_000, seed=1, return_stderr=False)
bs = op.bs_price(opt2)
print("MC:", mc, "BS:", bs, "rel err:", abs(mc-bs)/bs)


MC: 5.282522624386058 BS: 5.2933980580449 rel err: 0.0020545278362948936


### Monte Carlo vs Black-Scholes

With enough paths, the Monte Carlo price converges to the closed-form Black-Scholes value. The relative error should be well below 1%.

In [4]:
import optpricer as op
from dataclasses import replace, asdict

---

## 2. One Spec, Many Pricers

A core design principle: define the contract **once**, then pass it to any pricing engine. Below we compare three independent methods on the same ATM European call ($r = 5\%$, $\sigma = 20\%$, $T = 1$).

In [5]:
# 1) One spec, many pricers
# Define an option
opt = op.OptionSpec(S0=100, K=100, T=1.0, r=0.05, sigma=0.20, q=0.0)

In [6]:
op.bs_price(opt, op.CALL), op.bs_price(opt, op.PUT)

(10.450583572185565, 5.573526022256971)

Call and put prices under Black-Scholes. Note that put-call parity holds: $C - P = S_0 e^{-qT} - K e^{-rT}$.

In [7]:
print('Option premium')
print('Black-Scholes:', op.bs_price(opt, op.CALL))                  # closed-form
print('Monte-Carlo:', op.euro_price_mc(opt, op.CALL))             # Monte Carlo
print('Binomial Tree:', op.crr(opt, op.CALL, N=500, american=False))# Binomial Euro

Option premium
Black-Scholes: 10.450583572185565
Monte-Carlo: (10.477150737292655, 0.012542287557683876)
Binomial Tree: 10.44658513644654


All three engines should agree to within a few cents (MC noise and tree discretisation are the only sources of discrepancy).

In [8]:
# 2) Compute Greeks & implied vol
print(op.bs_greeks(opt))

# Implied volatility
iv = op.implied_vol(opt, target_price=10.45)  # solve sigma
print(iv)

{'delta': 0.6368306511756191, 'gamma': 0.018762017345846895, 'vega': 37.52403469169379, 'theta': -6.414027546438197, 'rho': 53.232481545376345}
0.19998444800547083


## 3. Greeks & Implied Volatility

`bs_greeks` returns $\Delta$, $\Gamma$, $\Theta$, $\mathcal{V}$ (vega), and $\rho$ in a single call. `implied_vol` inverts the BS formula via Brent's method to recover $\sigma$ from a market price.

In [9]:
# 3) Clone with a tweak (immutable-friendly)
# Based on the option object defined above, we make another object with a different strike K
opt_110 = replace(opt, K=110)
print(op.bs_price(opt_110))
print(op.bs_price(opt))

6.040088129724225
10.450583572185565


## 4. Immutable Clone-and-Tweak

Because `OptionSpec` is a frozen dataclass, we use `dataclasses.replace` to create a new spec with a modified strike. The original object is untouched.

In [10]:
# 4) Batch across strikes (a small vertical call strip)
strikes = [90, 95, 100, 105, 110]
strip = [(K, op.bs_price(replace(opt, K=K))) for K in strikes]
print(strip)

[(90, 16.69944840841599), (95, 13.346464945879582), (100, 10.450583572185565), (105, 8.021352235143176), (110, 6.040088129724225)]


## 5. Batch Pricing — A Vertical Call Strip

Looping over a list of strikes to build a call spread ladder. This is a common workflow for scanning an option chain.

In [11]:
# 5) Interop with dicts / JSON
d = asdict(opt)                 # {'S0':100, 'K':100, ...}
opt2 = op.OptionSpec(**d)       # reconstruct from dict

## 6. Interop with Dicts / JSON

`OptionSpec` round-trips cleanly through `asdict` and the constructor, making it straightforward to serialise for logging, REST APIs, or database storage.

In [12]:
opt

OptionSpec(S0=100, K=100, T=1.0, r=0.05, sigma=0.2, q=0.0)

In [13]:
import numpy as np
import optpricer as op
from optpricer.processes import (
    gbm_paths, merton_jump_paths, heston_paths, sabr_paths, local_vol_paths
)

# European price under Heston via MC
S = heston_paths(S0=100, r=0.02, q=0.0, v0=0.04, kappa=1.5, theta=0.04,
                 xi=0.6, rho=-0.7, T=1.0, n_steps=252, n_paths=50_000, seed=1)
ST = S[-1]
disc = np.exp(-0.02 * 1.0)
call = disc * np.maximum(ST - 100, 0).mean()
print("Heston MC call ~", call)

# Merton jump-diffusion
S = merton_jump_paths(S0=100, r=0.02, q=0.0, sigma=0.2, T=1.0, n_steps=252, n_paths=50_000,
                      lam=0.5, mJ=-0.1, sJ=0.2, seed=1)
ST = S[-1]
call = np.exp(-0.02) * np.maximum(ST - 100, 0).mean()

# SABR (β=1 behaves like stochastic-vol GBM)
S = sabr_paths(100, 0.02, 0.0, alpha0=0.2, beta=1.0, nu=1.0, rho=-0.5,
               T=1.0, n_steps=252, n_paths=50_000, seed=2)

# Local vol with a toy smile: σ(S,t) increasing away from ATM
def toy_lv(S, t):
    atm = 100.0
    base, slope = 0.18, 0.0015
    return base + slope * np.abs(S - atm)
S = local_vol_paths(100, r=0.02, q=0.0, T=1.0, n_steps=252, n_paths=20_000,
                    sigma_loc=toy_lv, seed=3)


Heston MC call ~ 8.104620965571499


---

## 7. Stochastic Process Engines

`optpricer.processes` provides five path generators. Each returns a matrix of shape `(n_steps+1, n_paths)`.

| Process | Key feature |
|---|---|
| **GBM** | Constant vol, exact log-Euler |
| **Merton Jump-Diffusion** | Poisson jumps with log-normal sizes |
| **Heston** | Stochastic variance with mean reversion |
| **SABR** | Stochastic vol with $\beta$-elasticity |
| **Local Vol** | Deterministic $\sigma(S,t)$ surface |

Below we price a European call under each model via Monte Carlo and show how to define a toy local-vol surface.